This notebook contains details about a potential code of conduct bug in the GitHub GraphQL API.

When getting information from the codeOfConduct object for all respositories in an org, it works find for some orgs while failing with an error message for other orgs. When it fails for an org, it neglects to get ANY data from the repository.

In [1]:
import requests
import json

# Assumes your GH API token is in a file called gh_key in this directory
with open('gh_key', 'r') as kf:
    api_token = kf.readline().rstrip() # remove newline & trailing whitespace

In [24]:
# fails on vmware, bitnami
# works on vmware-tanzu, concourse, carbonblack
def make_query():
    return """query RepoQuery($org_name: String!) {
             organization(login: $org_name) {
               repositories (first: 100){
                 pageInfo {
                   hasNextPage
                   endCursor
                 }
                 nodes {
                   name
                   codeOfConduct{
                     url
                   }
                   createdAt
                 }
                }
               }
            }"""

# Works fine for some orgs: vmware-tanzu example

In [25]:
# fails on vmware, bitnami
# works on vmware-tanzu, concourse, carbonblack
org_name = "vmware-tanzu"

In [26]:
url = 'https://api.github.com/graphql'
headers = {'Authorization': 'token %s' % api_token}

query = make_query()

variables = {"org_name": org_name}
r = requests.post(url=url, json={'query': query, 'variables': variables}, headers=headers)
json_data_tanzu = json.loads(r.text)
print(json_data_tanzu)

{'data': {'organization': {'repositories': {'pageInfo': {'hasNextPage': False, 'endCursor': 'Y3Vyc29yOnYyOpHOGL3pZg=='}, 'nodes': [{'name': 'sonobuoy', 'codeOfConduct': {'url': 'https://github.com/vmware-tanzu/sonobuoy/blob/main/CODE_OF_CONDUCT.md'}, 'createdAt': '2017-07-26T18:27:09Z'}, {'name': 'velero', 'codeOfConduct': {'url': 'https://github.com/vmware-tanzu/velero/blob/main/CODE_OF_CONDUCT.md'}, 'createdAt': '2017-08-02T17:22:11Z'}, {'name': 'velero-plugin-example', 'codeOfConduct': {'url': 'https://github.com/vmware-tanzu/velero-plugin-example/blob/main/CODE_OF_CONDUCT.md'}, 'createdAt': '2017-11-28T20:25:03Z'}, {'name': 'tgik', 'codeOfConduct': {'url': 'https://github.com/vmware-tanzu/tgik/blob/master/CODE-OF-CONDUCT.md'}, 'createdAt': '2018-05-07T18:11:06Z'}, {'name': 'carvel-kwt', 'codeOfConduct': {'url': 'https://github.com/vmware-tanzu/carvel-kwt/blob/develop/CODE_OF_CONDUCT.md'}, 'createdAt': '2018-09-24T17:59:19Z'}, {'name': 'thepodlets', 'codeOfConduct': {'url': 'https:/

# Fails on other orgs: vmware org example

In [27]:
# fails on vmware, bitnami
# works on vmware-tanzu, concourse, carbonblack
org_name = "vmware"

In [28]:
url = 'https://api.github.com/graphql'
headers = {'Authorization': 'token %s' % api_token}

query = make_query()

variables = {"org_name": org_name}
r = requests.post(url=url, json={'query': query, 'variables': variables}, headers=headers)
json_data_vmware = json.loads(r.text)
print(json_data_vmware)

{'data': None, 'errors': [{'message': 'Something went wrong while executing your query. This may be the result of a timeout, or it could be a GitHub bug. Please include `CAB0:C404:1B10EB1:1B93850:61E29378` when reporting this issue.'}]}


## Note: If you remove the codeOfConduct, the rest of the query works fine.

In [29]:
# fails on vmware, bitnami
# works on vmware-tanzu, concourse, carbonblack
def make_query_no_coc():
    return """query RepoQuery($org_name: String!) {
             organization(login: $org_name) {
               repositories (first: 100){
                 pageInfo {
                   hasNextPage
                   endCursor
                 }
                 nodes {
                   name
                   createdAt
                 }
                }
               }
            }"""

In [30]:
url = 'https://api.github.com/graphql'
headers = {'Authorization': 'token %s' % api_token}

query = make_query_no_coc()

variables = {"org_name": org_name}
r = requests.post(url=url, json={'query': query, 'variables': variables}, headers=headers)
json_data_no_coc = json.loads(r.text)
print(json_data_no_coc)

{'data': {'organization': {'repositories': {'pageInfo': {'hasNextPage': True, 'endCursor': 'Y3Vyc29yOnYyOpHOCThfGA=='}, 'nodes': [{'name': 'pg_rewind', 'createdAt': '2013-05-23T10:45:43Z'}, {'name': 'pyvmomi', 'createdAt': '2013-12-13T17:30:30Z'}, {'name': 'pyvmomi-community-samples', 'createdAt': '2014-04-24T20:31:56Z'}, {'name': 'open-vm-tools', 'createdAt': '2014-04-25T21:30:54Z'}, {'name': 'upgrade-framework', 'createdAt': '2014-06-16T17:22:11Z'}, {'name': 'workflowTools', 'createdAt': '2014-07-18T22:16:00Z'}, {'name': 'govmomi', 'createdAt': '2014-08-12T16:15:08Z'}, {'name': 'pyvcloud', 'createdAt': '2014-11-12T19:36:04Z'}, {'name': 'vmw-guestinfo', 'createdAt': '2014-11-29T23:07:44Z'}, {'name': 'vcd-cli', 'createdAt': '2014-12-05T18:52:29Z'}, {'name': 'open-vmdk', 'createdAt': '2014-12-15T17:10:11Z'}, {'name': 'tdnf', 'createdAt': '2015-02-26T00:44:11Z'}, {'name': 'likewise-open', 'createdAt': '2015-02-26T19:58:04Z'}, {'name': 'photon', 'createdAt': '2015-04-15T17:22:47Z'}, {'nam